# Testes & Check de Qualidade

Este notebook documenta a camada de **qualidade e reprodutibilidade** do projeto, demonstrando como executar uma suíte de testes automatizados com `pytest`.

A presença de testes, mesmo que mínimos, agrega valor ao case por:
- reduzir o risco de **data leakage** em problemas temporais;
- garantir consistência do **feature engineering** (evitando `NaN/inf` e colunas faltantes);
- validar o contrato das **métricas** usadas para comparar modelos;
- aumentar a confiança na manutenção do projeto (mudanças futuras sem “quebrar” o pipeline).


## Escopo dos testes

A suíte foi mantida propositalmente simples e focada em pontos críticos do case:

1. **Split temporal (anti-leakage)**
   - garante que o conjunto de treino ocorre antes do teste (ordem preservada)
   - valida tamanho do split e validações de parâmetros

2. **Feature engineering (robustez)**
   - valida que features derivadas existem (ex.: `total_malt`, `roast_pct`, `mt_energy`, `wk_energy`)
   - evita falhas comuns como divisão por zero e valores não finitos

3. **Métricas (consistência)**
   - garante que funções de avaliação retornam chaves e tipos consistentes (`mae`, `rmse`, `r2`)
   - evita erros silenciosos que impactam comparações e reporting


## Setup do ambiente (Colab / Google Drive)

Em ambientes como Google Colab, o *working directory* pode variar.  
Por isso, antes de executar o `pytest`, o diretório atual é ajustado (`CWD`) para apontar para a raiz do projeto.

Assim evita problemas de import (ex.: `ModuleNotFoundError: src`) e garante que os testes sejam executados no contexto correto.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
from pathlib import Path

# Acessando funções
current_path = Path.cwd()

if current_path == Path('/'):
    # Em caso do diretorio atual não ser o root,
    # PROJECT_ROOT é definido como "/content".
    PROJECT_ROOT = Path('/content')
    print(f"Atenção: Diretorio atual é o root ('/'). Definindo PROJECT_ROOT para {PROJECT_ROOT}.")
elif current_path.parents:
    # Caso contrário, obtenha o diretório origem conforme o esperado.
    PROJECT_ROOT = current_path.parents[0]
else:
   # Caso nenhuma consição atender usaremos o caminho atual por padrão neste cenário incomum.
    PROJECT_ROOT = current_path
    print(f"Warning: Could not determine parent for {current_path}. Setting PROJECT_ROOT to current path.")

os.chdir(PROJECT_ROOT)
print("CWD:", Path.cwd())

Atenção: Diretorio atual é o root ('/'). Definindo PROJECT_ROOT para /content.
CWD: /content


## Execução da suíte de testes

Abaixo executamos a suíte completa de testes localizada na pasta `tests/`.

O modo `-vv` (*verbose*) é útil para fins de documentação porque exibe:
- quais arquivos estão sendo executados
- quais testes passaram/falharam
- detalhes de cada caso de teste


In [8]:
!pytest -vv /content/drive/MyDrive/heineken_case/tests

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-4.12.0, langsmith-0.4.56, typeguard-4.4.4
collected 4 items                                                              

drive/MyDrive/heineken_case/tests/test_evaluation.py::test_eval_regression_returns_expected_keys_and_types PASSED [ 25%]
drive/MyDrive/heineken_case/tests/test_features.py::test_add_domain_features_creates_expected_columns PASSED [ 50%]
drive/MyDrive/heineken_case/tests/test_split.py::test_temporal_train_test_split_sizes_and_order PASSED [ 75%]
drive/MyDrive/heineken_case/tests/test_split.py::test_temporal_train_test_split_invalid_test_size PASSED [100%]

============================== 4 passed in 1.86s ===============================


## Conclusão

A inclusão de testes automatizados neste case fortalece a qualidade da entrega ao garantir:

- **reprodutibilidade** (mesmos resultados e validações ao reexecutar)
- **segurança contra leakage** (crítico em cenários temporais)
- **robustez de features** (evita falhas em dados futuros)
- **confiabilidade das métricas** (comparação correta entre modelos)

Com isso, o projeto fica mais próximo de um padrão de entrega aplicável em ambientes reais de produção, mesmo sendo um case técnico.
